#Imports e Dependências

In [2]:
import pandas as pd

##Obter datasets

In [1]:
!wget https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/countries.csv?raw=true

!wget https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/flags.csv?raw=true

!ls -la

--2021-02-09 00:22:29--  https://github.com/Andrewsas/trabalho-fundamentos-cid/blob/master/data/countries.csv?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Andrewsas/trabalho-fundamentos-cid/raw/master/data/countries.csv [following]
--2021-02-09 00:22:29--  https://github.com/Andrewsas/trabalho-fundamentos-cid/raw/master/data/countries.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Andrewsas/trabalho-fundamentos-cid/master/data/countries.csv [following]
--2021-02-09 00:22:29--  https://raw.githubusercontent.com/Andrewsas/trabalho-fundamentos-cid/master/data/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (ra

Rename files

In [3]:
!mv 'countries.csv?raw=true' 'countries.csv'

!mv 'flags.csv?raw=true' 'flags.csv'

In [4]:
!ls -la

total 60
drwxr-xr-x 1 root root  4096 Feb  9 00:24 .
drwxr-xr-x 1 root root  4096 Feb  9 00:21 ..
drwxr-xr-x 1 root root  4096 Feb  5 15:13 .config
-rw-r--r-- 1 root root 28446 Feb  9 00:22 countries.csv
-rw-r--r-- 1 root root 15456 Feb  9 00:22 flags.csv
drwxr-xr-x 1 root root  4096 Feb  4 15:26 sample_data


#Tratar datasets

In [5]:
df_countries = pd.read_csv('./countries.csv', sep=";")
df_flags = pd.read_csv('./flags.csv')

In [6]:
df_countries.head()

,alpha_2,alpha_3,area,capital,continent,currency_code,currency_name,eqivalent_fips_code,fips,geoname_id,languages,name,neighbours,numeric,phone,population,postal_code_format,postal_code_regex,tld
0,AD,AND,468.0,Andorra la Vella,EU,EUR,Euro,NaN,AN,3041565,ca,Andorra,"ES,FR",20,376,84000,AD###,^(?:AD)*(\d{3})$,.ad
1,AE,ARE,82880.0,Abu Dhabi,AS,AED,Dirham,NaN,AE,290557,"ar-AE,fa,en,hi,ur",United Arab Emirates,"SA,OM",784,971,4975593,NaN,NaN,.ae
2,AF,AFG,647500.0,Kabul,AS,AFN,Afghani,NaN,AF,1149361,"fa-AF,ps,uz-AF,tk",Afghanistan,"TM,CN,IR,TJ,PK,UZ",4,93,29121286,NaN,NaN,.af
3,AG,ATG,443.0,St. John's,NaN,XCD,Dollar,NaN,AC,3576396,en-AG,Antigua and Barbuda,NaN,28,+1-268,86754,NaN,NaN,.ag
4,AI,AIA,102.0,The Valley,NaN,XCD,Dollar,NaN,AV,3573511,en-AI,Anguilla,NaN,660,+1-264,13254,NaN,NaN,.ai


In [8]:
df_flags.head()

,name,landmass,zone,area,population,language,religion,bars,stripes,colours,red,green,blue,gold,white,black,orange,mainhue,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red


In [ ]:
# países comuns a estas duas fontes de informação
df_comum = pd.concat([df_flags, df_countries], ignore_index=True)
df_comum = df_comum.drop_duplicates()

df_comum = df_comum.sort_values(by=['name'])
df_comum

,name,landmass,zone,area,population,language,religion,bars,stripes,colours,red,green,blue,gold,white,black,orange,mainhue,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright,alpha_2,alpha_3,capital,continent,currency_code,currency_name,eqivalent_fips_code,fips,geoname_id,languages,neighbours,numeric,phone,postal_code_format,postal_code_regex,tld
0,Afghanistan,5.0,1.0,648.0,16,10.0,2.0,0.0,3.0,5.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,green,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,black,green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,Afghanistan,NaN,NaN,647500.0,29121286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AF,AFG,Kabul,AS,AFN,Afghani,NaN,AF,1149361.0,"fa-AF,ps,uz-AF,tk","TM,CN,IR,TJ,PK,UZ",4.0,93,NaN,NaN,.af
208,Aland Islands,NaN,NaN,1580.0,26711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AX,ALA,Mariehamn,EU,EUR,Euro,FI,NaN,661882.0,sv-AX,NaN,248.0,+358-18,#####,^(?:FI)*(\d{5})$,.ax
1,Albania,3.0,1.0,29.0,3,6.0,6.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,red,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,red,red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,Albania,NaN,NaN,28748.0,2986952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL,ALB,Tirana,EU,ALL,Lek,NaN,AL,783754.0,"sq,el","MK,GR,ME,RS,XK",8.0,355,NaN,NaN,.al
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Zaire,4.0,2.0,905.0,28,10.0,5.0,0.0,0.0,4.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,green,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,green,green,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,Zambia,4.0,2.0,753.0,6,10.0,5.0,3.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,green,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,green,brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,Zambia,NaN,NaN,752614.0,13460305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZM,ZMB,Lusaka,AF,ZMW,Kwacha,NaN,ZA,895949.0,"en-ZM,bem,loz,lun,lue,ny,toi","ZW,TZ,MZ,CD,NA,MW,AO",894.0,260,#####,^(\d{5})$,.zm
443,Zimbabwe,NaN,NaN,390580.0,13061000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZW,ZWE,Harare,AF,ZWL,Dollar,NaN,ZI,878675.0,"en-ZW,sn,nr,nd","ZA,MZ,BW,ZM",716.0,263,NaN,NaN,.zw


In [ ]:
# Quais países fazem parte apenas de flags.csv
df_only_in_flags = df_flags[df_flags.name.isin(df_countries.name)==False]
df_only_in_flags = df_only_in_flags.sort_values(by=['name'])

df_only_in_flags

,name,landmass,zone,area,population,language,religion,bars,stripes,colours,red,green,blue,gold,white,black,orange,mainhue,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright
3,American-Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
7,Antigua-Barbuda,1,4,0,0,1,1,0,1,5,1,0,1,1,1,1,0,red,0,0,0,0,1,0,1,0,0,0,black,red
9,Argentine,2,3,2777,28,2,0,0,3,3,0,0,1,1,1,0,0,blue,0,0,0,0,1,0,0,0,0,0,blue,blue
24,British-Virgin-Isles,1,4,0,0,1,1,0,0,6,1,1,1,1,1,0,1,blue,0,1,1,1,0,0,0,1,1,1,white,blue
27,Burkina,4,4,274,7,3,5,0,2,3,1,1,0,1,0,0,0,red,0,0,0,0,1,0,0,0,0,0,red,green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,USSR,5,1,22402,274,5,6,0,0,2,1,0,0,1,0,0,0,red,0,0,0,0,1,0,0,1,0,0,red,red
186,Vatican-City,3,1,0,0,6,0,2,0,4,1,0,0,1,1,1,0,gold,0,0,0,0,0,0,0,1,0,0,gold,white
189,Western-Samoa,6,3,3,0,1,1,0,0,3,1,0,1,0,1,0,0,red,0,0,0,1,5,0,0,0,0,0,blue,red
190,Yugoslavia,3,1,256,22,6,6,0,3,4,1,0,1,1,1,0,0,red,0,0,0,0,1,0,0,0,0,0,blue,red


In [ ]:
# Quais países fazem parte apenas de countries.csv
df_only_in_countries = df_countries[df_countries.name.isin(df_flags.name)==False]
df_only_in_countries = df_only_in_countries.sort_values(by=['name'])

df_only_in_countries

,alpha_2,alpha_3,area,capital,continent,currency_code,currency_name,eqivalent_fips_code,fips,geoname_id,languages,name,neighbours,numeric,phone,population,postal_code_format,postal_code_regex,tld
14,AX,ALA,1580.00,Mariehamn,EU,EUR,Euro,FI,NaN,661882,sv-AX,Aland Islands,NaN,248,+358-18,26711,#####,^(?:FI)*(\d{5})$,.ax
10,AS,ASM,199.00,Pago Pago,OC,USD,Dollar,NaN,AQ,5880801,"en-AS,sm,to",American Samoa,NaN,16,+1-684,57881,NaN,NaN,.as
8,AQ,ATA,14000000.00,NaN,AN,NaN,NaN,NaN,AY,6697173,NaN,Antarctica,NaN,10,NaN,0,NaN,NaN,.aq
3,AG,ATG,443.00,St. John's,NaN,XCD,Dollar,NaN,AC,3576396,en-AG,Antigua and Barbuda,NaN,28,+1-268,86754,NaN,NaN,.ag
6,AM,ARM,29800.00,Yerevan,AS,AMD,Dram,NaN,AM,174982,hy,Armenia,"GE,IR,AZ,TR",51,374,2968000,######,^(\d{6})$,.am
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,UZ,UZB,447400.00,Tashkent,AS,UZS,Som,NaN,UZ,1512440,"uz,ru,tg",Uzbekistan,"TM,AF,KG,TJ,KZ",860,998,27865738,######,^(\d{6})$,.uz
236,VA,VAT,0.44,Vatican City,EU,EUR,Euro,NaN,VT,3164670,"la,it,fr",Vatican,IT,336,379,921,#####,^(\d{5})$,.va
243,WF,WLF,274.00,Mata Utu,OC,XPF,Franc,NaN,WF,4034749,"wls,fud,fr-WF",Wallis and Futuna,NaN,876,681,16025,#####,^(986\d{2})$,.wf
65,EH,ESH,266000.00,El-Aaiun,AF,MAD,Dirham,NaN,WI,2461445,"ar,mey",Western Sahara,"DZ,MR,MA",732,212,273008,NaN,NaN,.eh
